In [ ]:
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant

In [ ]:
from google.colab import files
import pandas as pd
import csv
import nltk
import string
import numpy as np
import sys
from nltk.corpus import wordnet
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import torch #pytorch
import torch.nn as nn
import torch.optim as optim
import enchant

In [ ]:
!pip install -U torchdata

In [ ]:
!pip install torchtext

In [ ]:
import torch
from torch.utils import data
import enchant

class YelpData(torch.utils.data.Dataset):
  def __init__(self, dataset):
    self.dataset = dataset
    self.word_to_idx = {"<PAD>":0, "<UNK>": 1, "<END>": 2}
    self.indexes = []
    self.labels = []
    self.sen_to_idxs()


  def sen_to_idxs(self):

    d = enchant.Dict("en_US") 
    count = 3
    for i in range(len(self.dataset)):
      words = self.dataset[i][1].split(" ")

      for word in words:
        word = word.lower()
        if word and d.check(word):
          if word not in self.word_to_idx:
            self.word_to_idx[word] = count
            count += 1

    self.filtered_dict = {}
    for key, value in self.word_to_idx.items():
      if value >= 5:
        self.filtered_dict[key] = value

    self.word_to_idx = self.filtered_dict
        
    for i in range(len(self.dataset)):
      if self.dataset[i][0] == 2:
        self.labels.append(1)
      else:
        self.labels.append(0)
      sen = []
      words = self.dataset[i][1].split(" ")
      for word in words:
        word = word.lower()
        if word:
          if word in self.word_to_idx:
            sen.append(self.word_to_idx[word])
          else:
            sen.append(1)

      sen.append(2)
      sen = sen[:200]
      self.indexes.append(sen)
 
    for i in range(len(self.indexes)):
      while len(self.indexes[i]) < 200:
          self.indexes[i].append(0)
      
  def __len__(self):
    return len(self.indexes)
    
  def __getitem__(self, idx):
    return torch.LongTensor(self.indexes[idx]).squeeze(), torch.LongTensor([self.labels[idx]]).squeeze()


In [ ]:
import torchdata
import torchtext


data = torchtext.datasets.YelpReviewPolarity(root='.data', split='train')
data = list(data)
data = [(x[0], x[1]) for x in data]
train_data, test_data = data[:30000], data[30000:36000]

train_data_tensor = YelpData(train_data)
test_data_tensor = YelpData(test_data)


train_loader = torch.utils.data.DataLoader(train_data_tensor, batch_size=32, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data_tensor, batch_size=32, shuffle=True, num_workers=2, drop_last=True)


In [ ]:
class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.embedded_layer = nn.Embedding(vocab_size, embedding_dim, padding_idx = 0)
        self.lstm_layer = nn.LSTM(embedding_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
        embed_x = self.embedded_layer(x)
        lstm_x = self.lstm_layer(embed_x)[0]
        final_output = self.linear(lstm_x[:,-1])
        return final_output



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTM(128, 128, len(train_data_tensor.word_to_idx), 2).to(device)
loss_function = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()

for _ in range(10):
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        # Get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        predictions = model(inputs)
    
        loss = loss_function(predictions, labels)
        loss.backward()
        optimizer.step()


In [ ]:
torch.save(model, "drive/My Drive/lstm.pt")